In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model
from tensorflow.keras.layers import AveragePooling2D,Dropout,Dense,Dropout,Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import numpy as np

In [2]:
init_lr=1e-4
epochs=20
batch_sz=32

In [3]:
directory="dataset/"
categories=["with_mask","without_mask"]

In [4]:
data=[]
labels=[]

for category in categories:
    path=os.path.join(directory,category)
    for img in os.listdir(path):
        img_path=os.path.join(path,img)
        image=load_img(img_path,target_size=(224,224))
        image=img_to_array(image)
        image=preprocess_input(image)

        data.append(image)
        labels.append(category)

C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\PIL\Image.py:973: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


In [5]:
lb=LabelBinarizer()
labels=lb.fit_transform(labels)
labels=to_categorical(labels)

data=np.array(data,dtype="float32")
labels=np.array(labels)

x_train,x_test,y_train,y_test=train_test_split(data,labels,test_size=0.20,stratify=labels,random_state=42)

In [7]:
aug=ImageDataGenerator(rotation_range=20,zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,
                       shear_range=0.15,horizontal_flip=True,fill_mode="nearest")

In [8]:
basemodel=MobileNetV2(weights="imagenet",include_top=False,input_tensor=Input(shape=(224,224,3)))

In [9]:
from keras.layers import Flatten
headModel = basemodel.output
headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(128, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(2, activation="softmax")(headModel)

In [11]:
model=Model(inputs=basemodel.input,outputs=headModel)

for layers in basemodel.layers:
    layers.trainable=False

In [12]:
opt=Adam(lr=init_lr,decay=init_lr/epochs)
model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["accuracy"])

H = model.fit(
	aug.flow(x_train, y_train, batch_size=batch_sz),
	steps_per_epoch=len(x_train) // batch_sz,
	validation_data=(x_test, y_test),
	validation_steps=len(x_test) // batch_sz,
	epochs=epochs)

C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\optimizer_v2\optimizer_v2.py:355: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/20
95/95 [==============================] - 182s 2s/step - loss: 0.4722 - accuracy: 0.8072 - val_loss: 0.1770 - val_accuracy: 0.9817
Epoch 2/20
95/95 [==============================] - 119s 1s/step - loss: 0.1779 - accuracy: 0.9585 - val_loss: 0.0898 - val_accuracy: 0.9883
Epoch 3/20
95/95 [==============================] - 118s 1s/step - loss: 0.1135 - accuracy: 0.9726 - val_loss: 0.0602 - val_accuracy: 0.9922
Epoch 4/20
95/95 [==============================] - 121s 1s/step - loss: 0.0854 - accuracy: 0.9802 - val_loss: 0.0489 - val_accuracy: 0.9922
Epoch 5/20
95/95 [==============================] - 118s 1s/step - loss: 0.0779 - accuracy: 0.9779 - val_loss: 0.0428 - val_accuracy: 0.9935
Epoch 6/20
95/95 [==============================] - 119s 1s/step - loss: 0.0636 - accuracy: 0.9822 - val_loss: 0.0435 - val_accuracy: 0.9896
Epoch 7/20
95/95 [==============================] - 121s 1s/step - loss: 0.0572 - accuracy: 0.9842 - val_loss: 0.0350 - val_accuracy: 0.9935
Epoch 8/20
95

In [14]:
from sklearn.metrics import classification_report
predIdxs = model.predict(x_test, batch_size=batch_sz)

# for each image in the testing set we need to find the index of the
# label with corresponding largest predicted probability
predIdxs = np.argmax(predIdxs, axis=1)

# show a nicely formatted classification report
print(classification_report(y_test.argmax(axis=1), predIdxs,
	target_names=lb.classes_))

# serialize the model to disk
print("[INFO] saving mask detector model...")
model.save("my_mask_detector.model", save_format="h5")

              precision    recall  f1-score   support

   with_mask       0.99      0.99      0.99       383
without_mask       0.99      0.99      0.99       384

    accuracy                           0.99       767
   macro avg       0.99      0.99      0.99       767
weighted avg       0.99      0.99      0.99       767

[INFO] saving mask detector model...


C:\Users\user\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
